In [1]:
from sodapy import Socrata
import requests, zipfile, StringIO
import urllib2
import pandas as pd
import pypyodbc
import csv
import os
import tempfile

# Download PLUTO data
(for now, just extracting the Brooklyn file)

In [2]:
r = requests.get('http://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_15v1.zip')
pluto_zip = zipfile.ZipFile(StringIO.StringIO(r.content))

In [3]:
BK = pd.read_csv(pluto_zip.open('BK.csv'))

C:\Anaconda3\envs\py27\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (6,7,8,10,27,50,52,53,77,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
BK.head()

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,Version
0,BK,1,1,302,21,2000,13,33,11201,L118,...,196222,12d,,302 007,30101,,3000010001,11/26/2013,1,15v1
1,BK,1,2,302,21,2000,13,33,11201,L118,...,195992,12d,,300 000,30101,,3000010001,11/26/2013,1,15v1
2,BK,1,50,302,21,2000,13,33,11201,L118,...,195989,12d,,302 007,30101,E-231,0,,1,15v1
3,BK,3,1,302,21,3002,13,33,11201,L118,...,196151,12d,,302 007,30101,,0,,1,15v1
4,BK,3,5,302,21,,13,33,11201,L118,...,,12d,,302 007,30101,,0,,4,15v1


In [5]:
# Need to add a boroid as an integer, for joining to other sources
BK['boroid'] = 3

In [6]:
# check out a sample record (my neighbor)
BK.query('Block == 2112 & Lot == 51').values

array([['BK', 2112L, 51L, 302L, '35', '1002', '15', '35', '11217', 'L105',
        '88', '1200', '130 FT GREENE PLACE         ', 'R6B      ',
        '         ', '         ', '         ', '    ', '    ', '      ',
        '      ', '     ', 'R6B                        ',
        '                     ', 'N', 'B9', '1', 0L, ' ',
        'KENNETH Z. MANTELL AS', 1700L, 3440L, 0L, 3440L, 0L, 0L, 0L, 0L,
        0L, 0L, 7L, 1L, 3.0, 2L, 2L, 20.0, 85.0, 20.0, 40.0, '  ', '3',
        'N', '5', '1', 10661L, 36526L, 1610L, 1610L, 1899L, 'E', 0L, 0L,
        'Brooklyn Academy of Music               ',
        '                                   ', 2.02, 2.0, 0.0, 2.0, 3L,
        3021120051L, 0L, 35L, '990751', '189083', '16c', ' ', '302 035 ',
        '30706', '     ', '0', '          ', 1L, '15v1  ', 3L]], dtype=object)

# Get the Multiple Dwelling Registrations and join to PLUTO

In [7]:
client = Socrata("data.cityofnewyork.us", "JbH87Ju4WIAzc35ZUpMxLxFyM")

In [8]:
mdr = client.get("39m2-a2pz", content_type="csv", limit=2000)

In [9]:
mdrdf =pd.DataFrame(mdr)
mdrdf.columns = mdrdf.iloc[0]
mdrdf = mdrdf.reindex(mdrdf.index.drop(0))
mdrdf = mdrdf.convert_objects(convert_numeric=True)

C:\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [10]:
pd.merge(mdrdf, BK, left_on=['boroid','block','lot'], right_on=['boroid','Block','Lot'])

,bin,block,boro,boroid,buildingid,communityboard,highhousenumber,housenumber,lastregistrationdate,lot,...,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,Version
0,3067452,2692,BROOKLYN,3,309061,1,22,22,2015-10-06T00:00:00.000,49,...,203260,13a,,304 060,30904,,0,,1,15v1
1,3389513,3946,BROOKLYN,3,898323,5,2767,2767,2015-11-09T00:00:00.000,45,...,185775,17c,,316 012,31302,,3039460045,12/8/2006,1,15v1
2,3051566,1822,BROOKLYN,3,330759,3,188,188,2015-11-13T00:00:00.000,41,...,188616,17a,,305 015,30611,,0,,1,15v1
3,3164056,6303,BROOKLYN,3,177933,10,1367,1367,2015-11-20T00:00:00.000,43,...,163313,22b,,311 069,31904,,0,,1,15v1
4,3332526,1415,BROOKLYN,3,863599,9,373,373,2015-11-10T00:00:00.000,2,...,181729,17b,,307 062,30507,,3014150001,10/30/1990,1,15v1
5,3067163,2688,BROOKLYN,3,338530,1,170,170,2015-11-10T00:00:00.000,78,...,203480,13a,,304 050,30904,,0,,1,15v1
6,3035639,1340,BROOKLYN,3,938328,8,1552,1552,2015-11-13T00:00:00.000,17,...,185845,17a,,305 054,30505,,0,,1,15v1
7,3158652,6196,BROOKLYN,3,170076,11,2055,2055,2015-11-10T00:00:00.000,56,...,162003,22d,,312 062,31901,,0,,1,15v1
8,3119002,5168,BROOKLYN,3,226528,17,2424,2424,2015-11-04T00:00:00.000,17,...,173927,23a,,310 033,31603,,0,,1,15v1
9,3390905,5576,BROOKLYN,3,950968,11,6809,6809,2015-11-10T00:00:00.000,7501,...,164540,22d,,312 063,31704,,0,,1,15v1


# Read the Assessment rolls from DOF website zips

In [11]:
tc1_source = 'http://www1.nyc.gov/assets/finance/downloads/tar/tc1_16.zip'
r = requests.get(tc1_source)
tc1_zip = zipfile.ZipFile(StringIO.StringIO(r.content))
tc1 = tc1_zip.read('tc1.mdb')
tc1_zip.close()
len(tc1)

825106432

#### dump the extracted Access Database into a tempfile

In [12]:
temp = tempfile.NamedTemporaryFile(delete=False)
temp.write(tc1)
temp.seek(0)
temp.close()

#### Connect to it to query some data and dump to a csv file

In [13]:
# connect to db
conn_str = 'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + temp.name
con = pypyodbc.connect(conn_str)
cur = con.cursor()

In [14]:
# run a query and get the results 
SQL = 'SELECT BBLE FROM tc1;' # your query goes here
rows = cur.execute(SQL).fetchall()
cur.close()
con.close()

In [17]:
rows[:10]

[(u'1000750043'),
 (u'1000780040'),
 (u'1000970017'),
 (u'1000970036'),
 (u'1000970044'),
 (u'1000970045'),
 (u'1000970055'),
 (u'1000970144'),
 (u'1001060002'),
 (u'1001060006')]

In [18]:
# you could change the mode from 'w' to 'a' (append) for any subsequent queries
with open('tc1_bble.csv', 'wb') as fou:
    csv_writer = csv.writer(fou) # default field-delimiter is ","
    csv_writer.writerows(rows)

#### don't forget to delete the temp file!

In [19]:
os.remove(temp.name)